In [1]:
import os
import shutil
import numpy as np
import zarr
import yaml

from aicsimageio import AICSImage, writers
from ome_zarr.io import parse_url, ZarrLocation
from ome_zarr.writer import write_image

from matplotlib.pyplot import imshow, subplots
from tifffile import imwrite

import cv2

In [6]:
RTDIR = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/"
GTDIR = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/"
TRDIR = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/"
FILEWRITE = True

In [4]:
# utility functions
# define file path
def dataPath(
    dirPath: str = "/",
    fileName: str = None,
    fileID: int = 0,
    outputMode: int = 0  # output options: 0: file path, 1:splitted path, 2: file list
):

    fileList = os.listdir(dirPath)
    filteredList = [item for item in fileList if not os.path.isdir(os.path.join(dirPath, item))]
    fileList = filteredList
    
    if fileName is None:
        fileName = fileList[fileID]
    fileName, filExt = os.path.splitext(fileName)
    filePath = os.path.join(dirPath, fileName + filExt)

    assert outputMode in [0, 1, 2], "outputMode must be 0 or 1"
    match outputMode:
        case 0:  # return file path
            return filePath
        case 1:  # return file path and file extension
            return dirPath, fileName, filExt
        case 2:  # return names in a file directory
            return fileList
        case _:
            raise ValueError(
                "outputMode must be 0-2 (0: file path, 1:splitted path, 2: file list")


# load data into an AICSImage object
def dataLoader(filepath: str = None):
    assert filepath != None, "filepath is None"
    assert os.path.exists(filepath), "file does not exist"
    img = AICSImage(filepath)
    return img


# save file into a zarr dircetory
def newZarr(
    pathZarr: str = "/",
    img: AICSImage = None,
    remove: bool = True
):

    assert img != None, "img is None"
    if os.path.exists(pathZarr):
        if remove:
            shutil.rmtree(pathZarr)
        else:
            raise OSError("file exists")
    os.mkdir(pathZarr)
    store = parse_url(pathZarr, mode="w").store
    image = img.data
    root = zarr.group(store=store)
    axes = img.dims.order.lower()
    imgShape = img.shape
    write_image(
        image=image,
        group=root,
        axes=axes,
        storage_options=dict(chunks=(1, imgShape[3], imgShape[4]))
    )


# convert all files in a directory to zarr
def zarrConverter(
    path: str = "/",
    remove: bool = True,
    dataTypes: str = ".tif"
):

    assert os.path.exists(path), "file does not exist"

    fileList = dataPath(path, outputMode=2)
    assert len(fileList) > 0, "file directory is empty"

    filePaths = []
    for file in fileList:
        if file.endswith(dataTypes):
            filePath = dataPath(path, file)
            dirPath, fileName, _ = dataPath(path, file, outputMode=1)
            zarrPath = os.path.join(dirPath, (fileName + '.zarr'))
            filePaths.append(zarrPath)

            img = dataLoader(filePath)
            newZarr(
                pathZarr=zarrPath,
                img=img,
                remove=remove
            )
    assert len(filePaths) > 0, "no files are converted"
    return filePaths

def imshowPair(img1, img2):
    fig, ax = subplots(1, 2)
    ax[0].imshow(img1)
    ax[1].imshow(img2)

In [5]:
GTList = sorted(dataPath(GTDIR, outputMode=2))
TRList = sorted(dataPath(TRDIR, outputMode=2))

shortGTList = sorted([fileName[0:-5] for fileName in GTList])
shortTRList = sorted([fileName[0:-18] for fileName in TRList])

for i in range(len(GTList)):
    GTList[i] = shortGTList[i]+'.tiff'
    TRList[i] = shortTRList[i]+'_resliced_N=2.tiff'
    print(shortGTList[i][:20], shortTRList[i][:20])


c1_rec_CAM_1.hdf5.1. c1_rec_CAM_1.hdf5.1.
c2_rec_CAM_1.hdf5.0. c2_rec_CAM_1.hdf5.0.
c3_rec_CAM.hdf5.1.re c3_rec_CAM.hdf5.1.re
c3_rec_CAM_3.hdf5.0. c3_rec_CAM_3.hdf5.0.
c4_rec_CAM.hdf5.1.re c4_rec_CAM.hdf5.1.re
c5_rec_CAM_1.hdf5.1. c5_rec_CAM_1.hdf5.1.
c6_rec_CAM.hdf5.1.re c6_rec_CAM.hdf5.1.re
c7_rec_CAM_1.hdf5.1. c7_rec_CAM_1.hdf5.1.
c8_rec_CAM.hdf5.1.re c8_rec_CAM.hdf5.1.re
c9_rec_CAM.hdf5.1.re c9_rec_CAM.hdf5.1.re
c9_rec_CAM_1.hdf5.1. c9_rec_CAM_1.hdf5.1.
rec_18h22m07s_rec_CA rec_18h22m07s_rec_CA
rec_18h23m56s_rec_CA rec_18h23m56s_rec_CA
rec_18h24m38s_rec_CA rec_18h24m38s_rec_CA
rec_18h25m14s_rec_CA rec_18h25m14s_rec_CA
rec_18h26m29s_rec_CA rec_18h26m29s_rec_CA
rec_18h27m26s_rec_CA rec_18h27m26s_rec_CA
rec_18h29m19s_rec_CA rec_18h29m19s_rec_CA
rec_18h29m55s_rec_CA rec_18h29m55s_rec_CA
rec_18h30m42s_rec_CA rec_18h30m42s_rec_CA
rec_18h31m41s_rec_CA rec_18h31m41s_rec_CA
rec_18h32m12s_rec_CA rec_18h32m12s_rec_CA
rec_18h32m56s_rec_CA rec_18h32m56s_rec_CA
rec_18h33m48s_rec_CA rec_18h33m48s

In [9]:
path = GTDIR + 'EditUltimatePlusUltra/'
if not os.path.exists(path):
    os.mkdir(path)
else:
    print(f"Directory {path} already exists")

GTstack = []
for idx, fileName in enumerate(GTList):
    inputPath = os.path.join(GTDIR, fileName)
    outputPath = os.path.join(path, f"GT{idx}.tiff")
    
    if os.path.isfile(inputPath):
        img = AICSImage(inputPath)
        imgData = img.data.squeeze()[-2048:, :2048]
        GTstack.append(imgData)
        if FILEWRITE:
            if not os.path.exists(outputPath):
                imwrite(outputPath, imgData)
                print(f"File {outputPath} saved")
            else:
                print(f"File {outputPath} already exists")

File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT0.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT1.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT2.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT3.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT4.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT5.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT6.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT7.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/EditUltimatePlusUltra/GT8.tiff saved
F

In [10]:
path = TRDIR + 'EditUltimatePlusUltra/'
if not os.path.exists(path):
    os.mkdir(path)
else:
    print(f"Directory {path} already exists")

TRStack = []
for idx, fileName in enumerate(TRList):
    inputPath = os.path.join(TRDIR, fileName)
    outputPath = os.path.join(path, f"TR{idx}.tiff")
    
    if os.path.isfile(inputPath):
        img = AICSImage(inputPath)
        imgUp = cv2.resize(img.data.squeeze()[-1024:, :1024], (2048, 2048), interpolation=cv2.INTER_CUBIC)
        TRStack.append(imgUp)

        if FILEWRITE:
            if not os.path.exists(outputPath):
                imwrite(outputPath, imgUp)
                print(f"File {outputPath} saved")
            else:
                print(f"File {outputPath} already exists")


File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR0.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR1.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR2.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR3.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR4.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR5.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR6.tiff saved
File /mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/EditUltimatePlusUltra/TR7.tiff saved
File /mn

In [37]:
pathZarr = os.path.join(RTDIR + 'data.zarr')
f = zarr.open(pathZarr, mode='w')

TRZarr = np.stack(TRStack, axis=0)
GTZarr = np.stack(GTstack, axis=0)

f['tr'] = TRZarr
f['tr'].attrs['resolution'] = [1, 1, 1]
f['gt'] = GTZarr
f['gt'].attrs['resolution'] = [1, 1, 1]



In [14]:
pathZarr = os.path.join(RTDIR, 'dataC.zarr')
if os.path.exists(pathZarr):
    shutil.rmtree(pathZarr)
f = zarr.open(pathZarr, mode='w')
gt = f.create_group('gt')
tr = f.create_group('tr')
for idx, fileName in enumerate(GTList):
    gt.create_dataset(f'gt{idx:03}', data=GTstack[idx], chunks=(1024, 1024))
    gt[f'gt{idx:03}'].attrs['resolution'] = [1, 1]
    tr.create_dataset(f'tr{idx:03}', data=TRStack[idx], chunks=(1024, 1024))
    tr[f'tr{idx:03}'].attrs['resolution'] = [1, 1]
    print(f"File {idx} saved")

gt.attrs['resolution'] = [1, 1]
tr.attrs['resolution'] = [1, 1]

File 0 saved
File 1 saved
File 2 saved
File 3 saved
File 4 saved
File 5 saved
File 6 saved
File 7 saved
File 8 saved
File 9 saved
File 10 saved
File 11 saved
File 12 saved
File 13 saved
File 14 saved
File 15 saved
File 16 saved
File 17 saved
File 18 saved
File 19 saved
File 20 saved
File 21 saved
File 22 saved
File 23 saved
File 24 saved
File 25 saved
File 26 saved
File 27 saved
File 28 saved
File 29 saved
File 30 saved
File 31 saved
File 32 saved
File 33 saved
File 34 saved
File 35 saved
File 36 saved
File 37 saved
File 38 saved
File 39 saved
File 40 saved
File 41 saved
File 42 saved
File 43 saved
File 44 saved
File 45 saved
File 46 saved
File 47 saved
File 48 saved
File 49 saved
